In [1]:
!pip install gradio transformers torch sentencepiece -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94

In [2]:

import gradio as gr
import torch
from transformers import pipeline, T5ForConditionalGeneration, T5Tokenizer

# Load the BERT model for grammatical and coherence analysis
grammar_checker = pipeline("text-classification", model="textattack/bert-base-uncased-CoLA")

# Load the T5 model for feedback generation
t5_model_name = "t5-small"
t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_name)
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_name)

# Function to analyze grammar and coherence using BERT
def analyze_text(text):
    result = grammar_checker(text)
    feedback = result[0]['label']
    confidence = result[0]['score']

    if feedback == "LABEL_1":
        return "✅ The text is grammatically correct and well-structured."
    else:
        return f"⚠️ The text has grammatical issues. Confidence: {confidence:.2f}"

# Function to generate essay feedback using T5
def generate_feedback(text):
    input_text = "feedback: " + text
    inputs = t5_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = t5_model.generate(**inputs, max_length=150, num_return_sequences=1)
    feedback = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return feedback

# Combined function for Gradio UI
def essay_feedback(text):
    grammar_feedback = analyze_text(text)
    detailed_feedback = generate_feedback(text)
    return grammar_feedback, detailed_feedback

# Gradio Interface
iface = gr.Interface(
    fn=essay_feedback,
    inputs=gr.Textbox(lines=10, placeholder="Paste your essay here..."),
    outputs=[gr.Textbox(label="Grammar & Structure Feedback"), gr.Textbox(label="Detailed Writing Feedback")],
    title="AI-Powered Essay Feedback Tool",
    description="📖 Get instant feedback on your writing using AI-powered BERT and T5 models.",
    theme="compact"
)

# Launch Gradio app
iface.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1108: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-67c69b15-4e5538aa4832598354ef98c5;ec982b73-fc97-450f-93d0-ac3daa5bf8ef)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://05ea0ef18e5496128b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
!pip install gradio deploy deploynow

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 5.0 MB/s eta 0:00:00
  Created wheel for deploy: filename=deploy-1.9.1-py3-none-any.whl size=6491 sha256=72472e6ce2a1bc251b1dc3a12c14004b2f8945b6c3f11c7d90603a376967ff8e
  Stored in directory: /root/.cache/pip/wheels/4e/d1/1f/000b5094925eb9441f92aced2600750fb6f44cf01418b9e56b
  Created wheel for deploynow: filename=deploynow-1.10.0-py3-none-any.whl size=6507 sha256=a3a0db03afadd8d1c791904407d34b536295bfdf043641c0ef3e70bbbc1ca834
  Stored in directory: /root/.cache/pip/wheels/2e/c1/d8/1579efab40880ed6c0e56df9d6c4f1e6365bf4f139cae9f80c
Successfully built deploy deploynow
